In [1]:
from current_season_code.functions import *
import pandas as pd
import numpy as np

In [2]:
bets = pd.read_csv('current_season_data/bets.csv', index_col = 0)

In [3]:
bets[['Team1', 'Team2', 'Team1_KC', 'Team2_KC', 'Team1_Bet', 'Team2_Bet']]

,Team1,Team2,Team1_KC,Team2_KC,Team1_Bet,Team2_Bet
0,Pacers,Lakers,0.025597,0.00000,2546.462888,0.00000
1,Magic,Hornets,0.017174,0.00000,1708.470390,0.00000
2,Cavaliers,Suns,0.025120,0.00000,2498.934984,0.00000
3,Celtics,Nets,0.013419,0.00000,1334.983959,0.00000
4,Timberwolves,Heat,0.006395,0.00000,636.140183,0.00000
5,Rockets,Bulls,0.016663,0.00000,1657.658105,0.00000
6,Thunder,Jazz,0.023690,0.00000,2356.754012,0.00000
7,Pelicans,Wizards,0.002988,0.00000,0.000000,0.00000
8,Bucks,Pistons,0.044491,0.00344,0.000000,342.18361
9,Grizzlies,Raptors,0.014671,0.00000,1459.467731,0.00000


In [2]:
calculate_bet_results(99481)

Today's capital is 93016.83485397066
